Import Tensorflow and other libraries

In [3]:
import tensorflow as tf
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
from IPython import display

Download MNIST Dataset

In [4]:
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

In [5]:
print("Length of train images = ", len(train_images))
print("Length of train labels = ", len(train_labels))

Length of train images =  60000
Length of train labels =  60000


In [6]:
train_images.shape

(60000, 28, 28)

In [7]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')

In [8]:
train_images.shape

(60000, 28, 28, 1)